In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import sent_tokenize
import string
from sklearn.cluster import KMeans
import time
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import random
from itertools import combinations
nltk.download('wordnet')

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

device = torch.device('cuda:0' if torch.cuda.is_available() else 0)
model.to(device)

def make_summarization(sentences, tokenizer, model): # 문자열 생성 요약
    if(len(sentences) < 4): return [max(sentences, key=lambda x:len(x))]
    split = []
    for i in range(len(sentences)//8):
        split.append(sentences[:8])
        sentences = sentences[8:]

    for i in range(len(split)):
        if(len(sentences) == 0): break
        split[i].append(sentences.pop())
    
    if(len(sentences) != 0): split.append(sentences)
    
    split_sum = []
    for sentences in split:
        text = '\n'.join(sentences)
        start = time.time()
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        
        #model.generate(torch.tensor([input_ids[:1020]]).to(device), num_beams=3, max_length=256, eos_token_id=1)
        summary_ids = model.generate(torch.tensor([input_ids]).to(device),  num_beams=4,  max_length=256, min_length=10,  eos_token_id=1)
        print(f"{time.time()-start:.4f} sec")
        sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        sum_result = sent_tokenize(sum_result)[0]
        sum_result = delete_repeat_str(sum_result)
        print(sum_result)
        split_sum.append(sum_result)
        print(len(split), len(split_sum))
        print('-----------------------------------------------')
    
    if(len(split_sum)==1):
        return [split_sum[0]]
      
    return split_sum


def summarize_topic(document_df, topic_num, tokenizer, model): # df와 topic num을 넣으면 해당 topic num을 요약
    sentences = []
    numbers = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
        numbers.append(eval(t)[0])
    result = make_summarization(sentences, tokenizer, model)
    avg = sum(numbers)/len(numbers)
    return (avg, result)


def summarize_first_sentences(processed_sentences, tokenizer, model): # 문쟈열을 k-means로 토픽 별 분류(첫 문장)
    clusternum = 1
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print('===================================================')
    
    first_result = (sum_result[0][0], [min(sum_result[0][1], key=lambda x:len(x))])
    print(first_result)
    return [first_result]
    

def summarize_topk_sentences(processed_sentences, tokenizer, model): # 문쟈열을 k-means로 토픽 별 분류
    clusternum = max(len(processed_sentences)//7, 1)
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        print('---------------------------------------------------')
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print(summ)
        print('***************************************************')
        
    return sorted(sum_result, key= lambda x: x[0])


def get_clustered_df(sentences, clusternum):
    print(clusternum)
    
    document_df = pd.DataFrame()
    document_df['opinion_text'] = [str(t) for t in sentences]
    
    if(len(sentences) < 2):
        document_df['cluster_label'] = 0
        print('len document df', len(document_df))
        return document_df.sort_values(by=['cluster_label'])
    
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

    lemmar = WordNetLemmatizer()
    
    # 토큰화한 각 단어들의 원형들을 리스트로 담아서 반환
    def LemTokens(tokens):
        return [lemmar.lemmatize(token) for token in tokens]

    # 텍스트를 Input으로 넣어서 토큰화시키고 토큰화된 단어들의 원형들을 리스트로 담아 반환
    def LemNormalize(text):
        return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
    
    tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                            ngram_range=(1,2),
                            min_df=0.05, max_df=0.85)


    ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
    kmeans = KMeans(n_clusters=clusternum, max_iter=10000, random_state=42)
    cluster_label = kmeans.fit_predict(ftr_vect)
    
    # 군집화한 레이블값들을 document_df 에 추가하기
    document_df['cluster_label'] = cluster_label
    return document_df.sort_values(by=['cluster_label'])
    

def get_topic_sentences(df, clabel):
    lst = []
    for i,t in enumerate(df[df['cluster_label'] == clabel]['opinion_text']):
        print(i, t)
        lst.append(t)
    return lst 


def delete_similar(input_sentences):
    if(len(input_sentences) < 2):
        return input_sentences
    sorted_sentences = sorted(input_sentences, key=lambda x:x[1][::-1])
    prev_len = len(sorted_sentences)
    
    for i in range(5):
        prev = sorted_sentences[0]
        processed_sentences = []
        for j,sentence in enumerate(sorted_sentences[1:]):
            s1 = set(prev[1].split())
            s2 = set(sentence[1].split())
            actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
            if(actual_jaccard < 0.5): # if not similar
                processed_sentences.append(prev)
                prev = sentence
            else:
                print(prev)
                print(sentence)
                print(actual_jaccard)
                print('-------------------------------------------')
                
        s1 = set(prev[1].split())
        
        if(len(processed_sentences) == 0):
            processed_sentences.append(prev)
            return processed_sentences
        
        s2 = set(processed_sentences[-1][1].split())
        actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
        if(actual_jaccard < 0.5): # if not similar
            processed_sentences.append(prev)
        
        sorted_sentences = sorted(processed_sentences, key=lambda x:x[1])
        
        if(prev_len == len(sorted_sentences)):
            break
        prev_len = len(sorted_sentences)
        
    return sorted_sentences


def get_first_topk_sentences(df, topk_percent):
    first_sentences = []
    topk_sentences = []
    for a,b in zip(df['context2'], df['topk']):
        context = eval(str(a))
        topk = eval(str(b))
        k = int(len(topk)*(topk_percent/100))
        topk = topk[:k]
        
        first = []
        for item in topk:
            if(item[0] == 0): 
                first.append(item)
                
        if(len(first) == 0):
            first_sentences += [(0, context[0])]
            topk_sentences += topk
        else:
            first_sentences += first
            topk.remove(first[0])
            topk_sentences += topk
                
    print('before delete similar:', len(first_sentences), len(topk_sentences))
    first_sentences = delete_similar(first_sentences)
    topk_sentences = delete_similar(topk_sentences)
    print('after delete similar:', len(first_sentences), len(topk_sentences))
    return first_sentences, topk_sentences


def get_additional_topk_sentences(df, prev_topk_percent, next_topk_percent):
    topk_sentences = []
    next_topk_percent = prev_topk_percent + next_topk_percent
    for a,b in zip(df['context2'], df['topk']):
        context = eval(str(a))
        topk = eval(str(b))
        pk = int(len(topk)*(prev_topk_percent/100))
        k = int(len(topk)*(next_topk_percent/100))
        topk = topk[pk:k]
        topk_sentences += topk
                
    print('before delete similar:', len(topk_sentences))
    if(len(topk_sentences) == 0):
        return topk_sentences
    
    topk_sentences = delete_similar(topk_sentences)
    print('after delete similar:', len(topk_sentences))
    return topk_sentences    


def get_topk_sentences(k, user_input, model, tokenizer):
    bot_input_ids = News_to_input(user_input, openapi_key)
    
    chat_history_ids = summary(args, bot_input_ids, -1, '', None, model)
    pred_lst = list(chat_history_ids[0][:k])
    final_text = []
    for i,a in enumerate(user_input.split('.')):
        if i in pred_lst:
            final_text.append(a+'. ')
    return final_text
    


def delete_repeat(user_text):
    text = user_text.split()
    x = len(text)
    comb = list(combinations(range(x), 2))
    sorted_comb = sorted(comb, key=lambda x: x[1]-x[0], reverse = True)
    for item in sorted_comb:
        start, end = item
        if(end-start <= len(sorted_comb)/2 and end-start>2):
            find_str = ' '.join(text[start:end])
            rest_str = ' '.join(text[end:])
            idx = rest_str.find(find_str)
            if idx != -1:
                print('deleted :', idx, '|', find_str)
                print('ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ')
                ret = ' '.join(text[:end]) + ' ' + rest_str[:idx] + rest_str[idx+len(find_str)+1:]
                print(user_text)
                print('ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ')
                print(ret)
                print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
                return ret
    return user_text


def delete_repeat_str(user_text):
    prev = user_text
    new = delete_repeat(prev)
    while(new != prev):
        prev = new
        new = delete_repeat_str(prev)        
    return new


def calc_len(sum_result2):
    ret = 0
    for s in sum_result2:
        tmp = 0
        for c in s[1]:
            tmp += len(c)
        ret += tmp
    return ret

[nltk_data] Downloading package wordnet to /opt/ml/nltk_data...


In [24]:
import os
from os import listdir
from os.path import isfile, join, isdir

all_paths = [
 'bertopic(mpnet)',
 'bertopic(paraphrase)',
 'kmeans_12',
 'hdbscan',
 'kmeans_8',
 'bertopic(SR-BERT)',
 'kmeans_4']

i = 0
pathname = all_paths[i]
mypath = f'./topk/raws/{all_paths[i]}'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['bertopic_부동산_20221201_20221231_topk.csv',
 'bertopic_부동산_20221201_20221203_topk.csv',
 'bertopic_삼성전자_20221201_20221203_topk.csv',
 'bertopic_삼성전자_20221201_20221215_topk.csv',
 'bertopic_윤석열_20221201_20221215_topk.csv',
 'bertopic_부동산_20221201_20221215_topk.csv',
 'bertopic_윤석열_20221201_20221231_topk.csv',
 'bertopic_삼성전자_20221201_20221231_topk.csv',
 'bertopic_윤석열_20221201_20221203_topk.csv']

In [20]:
j = 0
name = onlyfiles[j].split('_topk')[0]
all_df = pd.read_csv(f'{mypath}/{onlyfiles[j]}')
print(len(all_df), name)
all_df.topic.unique()

315 bertopic_부동산_20221201_20221231


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55])

In [34]:
t = 50
print('Topic:', t)
topic_context_df = all_df[all_df.topic == t]
len(topic_context_df)

Topic: 50


7

In [35]:
start = time.time()
first_sentences, topk_sentences = get_first_topk_sentences(topic_context_df, 15)

sum_result1 = summarize_first_sentences(first_sentences, tokenizer, model)
sum_result2 = sum_result1 + summarize_topk_sentences(topk_sentences, tokenizer, model)

prev_topk = 15

threshold = min(570, len(topic_context_df) * 120)
while(calc_len(sum_result2) < threshold): # 너무 짧을 때
    print(f'.................{calc_len(sum_result2)}<{threshold} get additional topk {prev_topk} ...............')
    topk_sentences2 = get_additional_topk_sentences(topic_context_df, prev_topk, 7)
    if(len(topk_sentences2) > 0):
        sum_result3 = summarize_topk_sentences(topk_sentences2, tokenizer, model)
        sum_result2 = sorted(sum_result3 + sum_result2, key=lambda x:x[0])
    prev_topk += 7

final_result = ''
for v in sum_result2:
    paragraph = [s.strip() for s in v[1]]
    new_paragraph = [s if('.' in s[-2: ]) else s+'. ' for s in paragraph]
    final_result += ' '.join(new_paragraph)+'\n\n'


print('======================= Final result =========================')
print(final_result)
print(f"{time.time()-start:.4f} sec")
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
if not os.path.exists(f'./summary/{pathname}/'):
    os.makedirs(f'./summary/{pathname}/')
f = open(f'./summary/{pathname}/{name}_summary_{i}.txt', 'w')
f.write(final_result)
f.close()

before delete similar: 7 9
(0, '이도(YIDO)는 관계사인 이도캐피탈자산운용이 금융위원회로부터 일반 사모집합투자업으로 등록돼 인가를 마쳤다고 26일 밝혔다. ')
(0, '이도(YIDO)는 관계사인 이도캐피탈자산운용이 금융위원회로부터 일반 사모집합투자업으로 인가를 마치고 본격 출범한다고 26일 밝혔다. ')
0.6428571428571429
-------------------------------------------
(3, ' 이를 통해 투자자에게는 부동산의 전통 수익 구조인 안정적인 배당은 물론 가치상승에 따른 capital gain(자본이익)까지 얻을 수 있는 기회를 제공하게 된다. ')
(3, ' 이를 통해 투자자에게는 부동산의 전통 수익 구조인 안정적인 배당은 물론 가치상승에 따른 capital gain(자본이익)까지 얻을 수 있는 기회를 제공하게 된다. ')
1.0
-------------------------------------------
after delete similar: 6 8
1
0 (0, '"저평가된 자산 적극적으로 발굴해 가치 극대화"다. ')
1 (0, '금리인상과 고물가로 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행한다. ')
2 (0, '부동산 대체투자 전문 자산운용사가 인가를 마치고 새롭게 출범한다. ')
3 (0, '이 기사는 12월 27일 10:09 자본 시장의 혜안 “마켓인사이트”에 게재된 기사입니다. ')
4 (0, '이도(YIDO)는 관계사인 이도캐피탈자산운용이 금융위원회로부터 일반 사모집합투자업으로 등록돼 인가를 마쳤다고 26일 밝혔다. ')
5 (0, '체험형 인턴, 청년구직자에 전문 분야 직무 경험 제공다. ')


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.5866 sec
deleted : 0 | 내년 경기가 악화할 것이란 전망 속에서
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행하여 내년 경기가 악화할 것이란 전망 속에서 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도가 업계 최고 수준의 체험형 인턴을 시행한다.
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행하여 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도가 업계 최고 수준의 체험형 인턴을 시행한다.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
deleted : 50 | 업계 최고 수준의 체험형 인턴을
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행하여 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도가 업계 최고 수준의 체험형 인턴을 시행한다.
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행하여 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도가 시행한다.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행하여 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도가 시행한다.
1 1
-----------------------------------------------
(0.0, ['주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행하여 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도가 시행한다.'])
1
0 (8, ' 2023년 체험형 인턴 1기는 내년 1월 15일까지 서류 접수가 이뤄지며 서류전형과 면접을 거쳐 오는 2023년 2월 4일 최종 합격자를 발표한다. ')

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.3850 sec
2023년 체험형 인턴 1기는 내년 1월 15일까지 서류 접수가 이뤄지며 서류전형과 면접을 거쳐 오는 2023년 2월 4일 최종 합격자를 발표한다.
1 1
-----------------------------------------------
(6.625, ['2023년 체험형 인턴 1기는 내년 1월 15일까지 서류 접수가 이뤄지며 서류전형과 면접을 거쳐 오는 2023년 2월 4일 최종 합격자를 발표한다.'])
***************************************************
.................170<570 get additional topk 15 ...............
before delete similar: 7
after delete similar: 7
1
0 (13, ' <ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제( 무단전재 배포금지>다. ')
1 (4, ' 아울러 이도는 전문 운영 역량을 통한 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줌으로써 선진 경영 방식인 소유와 경영을 분리해 서로 견제 및 상호 보완하는 발전적인 경영방식을 이도캐피탈자산운용은 추구할 방침이다. ')
2 (7, ' 업무는 2023년 3월 2일부터 6월 30일까지 4개월간 서울 중구 본사 씨티스퀘어 및 각 현장에서 이뤄진다. ')
3 (4, ' 이도는 전문 운영 역량을 통한 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줄 방침이다. ')
4 (6, ' 이도는 프라임 오피스와 상업시설에 대한 자산관리(PM)·임대관리(LM)·시설관리(FM) 등을 아우르는 부동산 종합운영관리 서비스를 제공하고 있다. ')
5 (5, ' 이를 통해 투자자에게 안정적인 배당을 제공하고, 가치상승에 따른 자본이익까지 얻을 수 있는 기회를 제공하겠다는 게 회사 측의 목표다. ')
6 (10, ' 최정훈 ㈜이도 대표이사는 “금번 운용사 설립으로 부동산 개발 초기 투자는 물론 안정적인 자산에 대한 

...

0.7923 sec
이도는캐피탈자산운용은 2023년 3월 2일부터 6월 30일까지 4개월간 서울 중구 본사 씨티스퀘어 및 각 현장에서 전문 운영 역량을 통한 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줌으로써 선진 경영 방식인 소유와 경영을 분리해 서로 견제 및 상호 보완하는 발전적인 경영방식을 추구할 방침이다.
1 1
-----------------------------------------------
(7.0, ['이도는캐피탈자산운용은 2023년 3월 2일부터 6월 30일까지 4개월간 서울 중구 본사 씨티스퀘어 및 각 현장에서 전문 운영 역량을 통한 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줌으로써 선진 경영 방식인 소유와 경영을 분리해 서로 견제 및 상호 보완하는 발전적인 경영방식을 추구할 방침이다.'])
***************************************************
.................349<570 get additional topk 22 ...............
before delete similar: 7
after delete similar: 7
1
0 (8, ' 서울 도심에 있는 프라임 오피스 씨티스퀘어의 공실률을 4개월만에 100%에서 0%로 낮추기도 했다. ')
1 (4, ' 선진 경영 방식인 소유와 경영의 분리를 통해 서로 견제하고, 상호보완하는 발전적인 경영방식을 추구한다. ')
2 (4, ' 아울러 이도는 전문 운영 역량을 통한 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줌으로써 선진 경영 방식인 소유와 경영의 분리하여 서로 견제 및 상호 보완하는 발전적인 경영방식을 이도캐피탈자산운용은 추구할 방침이다. ')
3 (7, ' 앞서 이도는 2021년 체험형 인턴 급여로 월 300만원을 지급해 청년 구직자에게 큰 관심을 받았다. ')
4 (6, ' 이도는 전문적인 운영 역량을 통해 안정적인 운영 수익권을 얻고, 자산 소

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.8798 sec
서울 도심에 있는 프라임 오피스 씨티스퀘어의 공실률을 4개월만에 100%에서 0%로 낮추기도 한 이도캐피탈자산운용은 전문 운영 역량을 통해 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줌으로써 선진 경영 방식인 소유와 경영의 분리하여 서로 견제 및 상호 보완하는 발전적인 경영방식을 추구할 방침이다.
1 1
-----------------------------------------------
(6.571428571428571, ['서울 도심에 있는 프라임 오피스 씨티스퀘어의 공실률을 4개월만에 100%에서 0%로 낮추기도 한 이도캐피탈자산운용은 전문 운영 역량을 통해 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줌으로써 선진 경영 방식인 소유와 경영의 분리하여 서로 견제 및 상호 보완하는 발전적인 경영방식을 추구할 방침이다.'])
***************************************************
.................530<570 get additional topk 29 ...............
before delete similar: 4
after delete similar: 4
1
0 (6, ' 2023년 체험형 인턴 1기는 2023년 1월 15일까지 서류 접수가 진행되며, 서류전형과 면접을 거쳐 오는 2023년 2월 4일 최종 합격자가 발표된다. ')
1 (8, ' 이도는 국내에서 프라임오피스 및 상업시설에 대한 PM(자산관리)·LM(임대관리)·FM(시설관리) 등 부동산 종합 운영관리 서비스를 제공한다. ')
2 (7, ' 이도는 프라임오피스와 상업시설에 대한 자산관리(PM)·임대관리(LM)·시설관리(FM) 등을 아우르는 부동산 종합운영관리 서비스를 제공하고 있다. ')
3 (5, ' 체험형 인턴의 급여는 대기업 대졸 신입사원보다 높은 수준인 월 300만원으로 업계 최고 수준이다. ')
-----------------------------------

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


0.8367 sec
국내에서 프라임오피스 및 상업시설에 대한 PM(자산관리)·LM(임대관리)·FM(시설관리) 등 부동산 종합 운영관리 서비스를 제공하는 이도는 프라임오피스와 상업시설에 대한 자산관리(PM)·임대관리(LM)·시설관리(FM) 등을 아우르는 부동산 종합운영관리 서비스를 제공하고 있다.
1 1
-----------------------------------------------
(6.5, ['국내에서 프라임오피스 및 상업시설에 대한 PM(자산관리)·LM(임대관리)·FM(시설관리) 등 부동산 종합 운영관리 서비스를 제공하는 이도는 프라임오피스와 상업시설에 대한 자산관리(PM)·임대관리(LM)·시설관리(FM) 등을 아우르는 부동산 종합운영관리 서비스를 제공하고 있다.'])
***************************************************
======================= Final result =========================
주식회사 이도(YIDO)가 업계 최고 수준의 체험형 인턴을 시행하여 내년 경기가 악화할 것이란 전망 속에서 청년 일자리 창출을 위해 주식회사 이도가 시행한다.

국내에서 프라임오피스 및 상업시설에 대한 PM(자산관리)·LM(임대관리)·FM(시설관리) 등 부동산 종합 운영관리 서비스를 제공하는 이도는 프라임오피스와 상업시설에 대한 자산관리(PM)·임대관리(LM)·시설관리(FM) 등을 아우르는 부동산 종합운영관리 서비스를 제공하고 있다.

서울 도심에 있는 프라임 오피스 씨티스퀘어의 공실률을 4개월만에 100%에서 0%로 낮추기도 한 이도캐피탈자산운용은 전문 운영 역량을 통해 안정적인 운영 수익권을 얻고, 자산 소유권에 대한 수익권은 투자자에게 줌으로써 선진 경영 방식인 소유와 경영의 분리하여 서로 견제 및 상호 보완하는 발전적인 경영방식을 추구할 방침이다.

2023년 체험형 인턴 1기는 내년 1월 15일까지 서류 접수가 이뤄지며 서류전형과 면접을 거쳐 오는 2023년 2월 4일 최종 